<img src="images/dask_horizontal.svg" align="right" width="30%">

# 包：半结构化数据的并行列表：

Dask-bag擅长处理可以表示为任意输入序列的数据。我们将其称为 "混乱 "数据，因为它可以包含复杂的嵌套结构、缺失的字段、数据类型的混合物等。这种*函数式的编程风格很适合标准的Python迭代，比如可以在`itertools`模块中找到。

在数据处理流水线的初期，当大量的原始数据被首次消耗时，经常会遇到混乱的数据。初始数据集可能是JSON、CSV、XML或任何其他不执行严格结构和数据类型的格式。
出于这个原因，最初的数据群发和处理通常使用Python的`list`s、`dict`s和`set`s来完成。

这些核心数据结构是为通用存储和处理而优化的。 用迭代器/生成器表达式或像`itertools`或[`toolz`](https://toolz.readthedocs.io/en/latest/)这样的库添加流式计算，可以让我们在小空间里处理大量的数据。 如果我们将其与并行处理结合起来，那么我们可以搅动相当数量的数据。

Dask.bag是一个高级的Dask集合，用于自动化这种形式的常见工作负载。 简而言之

    dask.bag = map、filter、toolz +并行执行

**相关文档**：

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## 创建数据

In [ ]:
%run prep.py -d accounts

## 设置

同样，我们将使用分布式调度器。调度器将在[后面](05_distributed.ipynb)进行深入解释。

In [ ]:
from dask.distributed import Client

client = Client(n_workers=4)

## 创建

可以从Python序列、从文件、从S3上的数据等创建一个`Bag`。
我们演示使用`.take()`来显示数据的元素。(执行`.take(1)`的结果是一个有一个元素的元组)

请注意，数据被分割成块，每个块有很多项。在第一个例子中，两个分区各包含5个元素，在下面的两个例子中，每个文件被分割成一个或多个字节块。

In [ ]:
# 每个元素都是一个整数
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

In [ ]:
# 每个元素是一个文本文件，其中每行是一个JSON对象。
# 注意，压缩是自动处理的
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

In [ ]:
# 编辑sources.py来配置源位置
import sources
sources.bag_url

In [ ]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})
b.take(1)

## 操作

`Bag`对象拥有Python标准库、`toolz`或`pyspark`等项目中的标准功能API，包括`map`、`filter`、`groupby`等。

对`Bag`对象的操作会创建新的袋子。 调用`.compute()`方法来触发执行，就像我们对`Delayed`对象的操作一样。

In [ ]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

In [ ]:
# 阻断形式：等待完成(在这种情况下是非常快的)
c.compute()

### 例子: 账户JSON数据

我们已经在你的数据目录中创建了一个gzipped JSON数据的假数据集。 这就像我们稍后看到的 "DataFrame "示例中使用的例子一样，只是它将每个单独的 "id "的所有内容捆绑成了一条记录。 这类似于你可能从文档存储数据库或Web API中收集的数据。

每一行都是一个JSON编码的字典，其键如下

* id：客户的唯一标识符
* 名称：客户名称
* 交易：`transaction-id'、`amount'对的清单，该文件中客户的每一笔交易都有一个。

In [ ]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

我们的数据以文本行的形式从文件中出来。注意，文件解压是自动发生的。我们可以通过将`json.loads`函数映射到我们的袋子上，让这个数据看起来更合理。

In [ ]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

### 基本查询

一旦我们将JSON数据解析成合适的Python对象(`dict`s，`list`s等)，我们就可以通过创建小的Python函数在我们的数据上运行来执行更有趣的查询。

In [ ]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

In [ ]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

In [ ]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

In [ ]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [ ]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .take(3))

In [ ]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

In [ ]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

In [ ]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

### Groupby和Foldby

我们经常想通过一些函数或键对数据进行分组。 我们可以使用`.groupby`方法来实现这一目标，该方法简单明了，但会强制对数据进行完全洗牌（成本很高），也可以使用更难使用但更快的`.foldby`方法，该方法将groupby和reduction结合在一起进行流式处理。

* `groupby`。 洗牌数据，使所有键值相同的项都在同一个键值对中。
* `foldby`。 遍历数据，按键累积结果。

*注意：完整的groupby特别糟糕。在实际工作中，你最好使用 "foldby"，或尽可能改用 "DataFrame"。

### `groupby`

Groupby收集集合中的项目，使所有在某个函数下具有相同值的项目被收集在一起，成为一个键值对。

In [ ]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # 根据名字的长度分组

In [ ]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

In [ ]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

### `foldby`

Foldby一开始可能很奇怪。 它与其他库的以下函数类似。

* [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
* [`pyspark.RDD.combinedByKey'](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

当使用 "foldby "时，您提供的是

1.  对要素进行分组的关键功能
2.  一个二进制运算符，比如你会传递给`reduce`，你用来对每组进行还原。
3.  组合二元运算符，可以将数据集不同部分的两次`reduce`调用的结果组合起来。

你的还原必须是关联性的，它将在你的数据集的每个分区中并行发生。 它将在你的数据集的每个分区中并行发生。 然后，所有这些中间结果将由`combine`二进制操作符合并。

In [ ]:
b.foldby(lambda x: x % 2, binop=max, combine=max).compute()

### 带账户数据的示例

我们发现同名的人数。

In [ ]:
%%time
# 警告，这个需要一段时间... ...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

In [ ]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot + 1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

### 练习：计算每个名字的总金额

我们想把`name`键进行分组，然后把每个名字的金额加起来。

步骤

1.  创建一个小函数，给定一个像{'name': 'Alice', 'transactions': [{'金额': 1, 'id': 123}, {'金额': 2, 'id': 456}]}产生金额的总和，例如：`3`。

2.  稍微改变上面`foldby`例子的二进制运算符，使二进制运算符不计算条目数，而是累计金额之和。

In [ ]:
# Your code here...

## DataFrames

出于同样的原因，Pandas通常比纯Python快，`dask.dataframe`可以比`dask.bag`快。 后面我们会更多地使用DataFrames，但从Bag的角度来看，它经常是数据摄取中 "混乱 "部分的终点--一旦数据可以做成数据框架，那么复杂的拆分-应用-合并逻辑将变得更加直接和高效。

你可以用`to_dataframe`方法将一个简单的元组或平面字典结构的袋子转化为`dask.dataframe`。

In [ ]:
df1 = js.to_dataframe()
df1.head()

现在，这看起来就像一个定义良好的DataFrame，我们可以有效地对它进行类似Pandas的计算。

使用Dask DataFrame，我们事先计算同名同姓的人数需要多长时间？ 事实证明，`dask.dataframe.groupby()`比`dask.bag.groupby()`强了不止一个数量级；但是，它仍然无法与`dask.bag.foldby()`相提并论。

In [ ]:
%time df1.groupby('name').id.count().compute().head()

### 非正常化

这种DataFrame格式不太理想，因为`transactions`列充满了嵌套的数据，所以Pandas必须恢复到`object`dtype，这在Pandas中是相当慢的。 理想的情况是，我们只有在将数据扁平化，使每条记录都是一个单一的 "int"、"string"、"float "等之后，才想转换为数据框架。

In [ ]:
def denormalize(record):
    # returns a list for each person, one item per transaction
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

In [ ]:
df = transactions.to_dataframe()
df.head()

In [ ]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

## 局限

袋子提供了非常通用的计算(任何Python函数。)这种通用性是指
是有成本的。 袋子有以下已知的限制

1.  袋式运算往往比数组/数据框计算慢，在这种情况下，袋式运算的速度会更快。
    就像Python比NumPy/Pandas慢一样。
2.  ``Bag.groupby``很慢。 如果可能的话，你应该尝试使用``Bag.foldby``。
    使用``Bag.foldby``需要更多的思考。更好的做法是，考虑创建
    归一化数据框。

## 更多信息，参考：

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## 关闭

In [ ]:
client.shutdown()